In [1]:
# Importing the libraries
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD

train_df = pd.read_csv("./input/train.csv")
test_df = pd.read_csv("./input/test.csv")

In [2]:
# Clean text
from tqdm import tqdm
tqdm.pandas()
punctuation = ['.', '..', '...', ',', ':', ';', '-', '*', '"', '!', '?']
alphabet = 'abcdefghijklmnopqrstuvwxyz'
def clean_text(x):
    x.lower()
    for p in punctuation:
        x.replace(p, '')
    return x

def extract_features(df, train_flag=False):
    df['text_cleaned'] = df['text'].apply(lambda x: clean_text(x))
    df['n_.'] = df['text'].str.count('\.')
    df['n_...'] = df['text'].str.count('\...')
    df['n_,'] = df['text'].str.count('\,')
    df['n_:'] = df['text'].str.count('\:')
    df['n_;'] = df['text'].str.count('\;')
    df['n_-'] = df['text'].str.count('\-')
    df['n_?'] = df['text'].str.count('\?')
    df['n_!'] = df['text'].str.count('\!')
    df['n_\''] = df['text'].str.count('\'')
    df['n_"'] = df['text'].str.count('\"')

    # First words in a sentence
    df['n_The '] = df['text'].str.count('The ')
    df['n_I '] = df['text'].str.count('I ')
    df['n_It '] = df['text'].str.count('It ')
    df['n_He '] = df['text'].str.count('He ')
    df['n_Me '] = df['text'].str.count('Me ')
    df['n_She '] = df['text'].str.count('She ')
    df['n_We '] = df['text'].str.count('We ')
    df['n_They '] = df['text'].str.count('They ')
    df['n_You '] = df['text'].str.count('You ')
    df['n_the'] = df['text_cleaned'].str.count('the ')
    df['n_ a '] = df['text_cleaned'].str.count(' a ')
    df['n_appear'] = df['text_cleaned'].str.count('appear')
    df['n_little'] = df['text_cleaned'].str.count('little')
    df['n_was '] = df['text_cleaned'].str.count('was ')
    df['n_one '] = df['text_cleaned'].str.count('one ')
    df['n_two '] = df['text_cleaned'].str.count('two ')
    df['n_three '] = df['text_cleaned'].str.count('three ')
    df['n_ten '] = df['text_cleaned'].str.count('ten ')
    df['n_is '] = df['text_cleaned'].str.count('is ')
    df['n_are '] = df['text_cleaned'].str.count('are ')
    df['n_ed'] = df['text_cleaned'].str.count('ed ')
    df['n_however'] = df['text_cleaned'].str.count('however')
    df['n_ to '] = df['text_cleaned'].str.count(' to ')
    df['n_into'] = df['text_cleaned'].str.count('into')
    df['n_about '] = df['text_cleaned'].str.count('about ')
    df['n_th'] = df['text_cleaned'].str.count('th')
    df['n_er'] = df['text_cleaned'].str.count('er')
    df['n_ex'] = df['text_cleaned'].str.count('ex')
    df['n_an '] = df['text_cleaned'].str.count('an ')
    df['n_ground'] = df['text_cleaned'].str.count('ground')
    df['n_any'] = df['text_cleaned'].str.count('any')
    df['n_silence'] = df['text_cleaned'].str.count('silence')
    df['n_wall'] = df['text_cleaned'].str.count('wall')
    
    new_df = df.copy()
    # Find numbers of different combinations
    for c in tqdm(alphabet.upper()):
        new_df['n_' + c] = new_df['text'].str.count(c)
        new_df['n_' + c + '.'] = new_df['text'].str.count(c + '\.')
        new_df['n_' + c + ','] = new_df['text'].str.count(c + '\,')

        for c2 in alphabet:
            new_df['n_' + c + c2] = new_df['text'].str.count(c + c2)
            new_df['n_' + c + c2 + '.'] = new_df['text'].str.count(c + c2 + '\.')
            new_df['n_' + c + c2 + ','] = new_df['text'].str.count(c + c2 + '\,')

    for c in tqdm(alphabet):
        new_df['n_' + c + '.'] = new_df['text'].str.count(c + '\.')
        new_df['n_' + c + ','] = new_df['text'].str.count(c + '\,')
        new_df['n_' + c + '?'] = new_df['text'].str.count(c + '\?')
        new_df['n_' + c + ';'] = new_df['text'].str.count(c + '\;')
        new_df['n_' + c + ':'] = new_df['text'].str.count(c + '\:')

        for c2 in alphabet:
            new_df['n_' + c + c2 + '.'] = new_df['text'].str.count(c + c2 + '\.')
            new_df['n_' + c + c2 + ','] = new_df['text'].str.count(c + c2 + '\,')
            new_df['n_' + c + c2 + '?'] = new_df['text'].str.count(c + c2 + '\?')
            new_df['n_' + c + c2 + ';'] = new_df['text'].str.count(c + c2 + '\;')
            new_df['n_' + c + c2 + ':'] = new_df['text'].str.count(c + c2 + '\:')
            new_df['n_' + c + ', ' + c2] = new_df['text'].str.count(c + '\, ' + c2)

    # And now starting processing of cleaned text
    for c in tqdm(alphabet):
        new_df['n_' + c] = new_df['text_cleaned'].str.count(c)
        new_df['n_' + c + ' '] = new_df['text_cleaned'].str.count(c + ' ')
        new_df['n_' + ' ' + c] = new_df['text_cleaned'].str.count(' ' + c)

        for c2 in alphabet:
            new_df['n_' + c + c2] = new_df['text_cleaned'].str.count(c + c2)
            new_df['n_' + c + c2 + ' '] = new_df['text_cleaned'].str.count(c + c2 + ' ')
            new_df['n_' + ' ' + c + c2] = new_df['text_cleaned'].str.count(' ' + c + c2)
            new_df['n_' + c + ' ' + c2] = new_df['text_cleaned'].str.count(c + ' ' + c2)

            for c3 in alphabet:
                new_df['n_' + c + c2 + c3] = new_df['text_cleaned'].str.count(c + c2 + c3)
                
    if train_flag:
        new_df.drop(['text_cleaned','text','author','id'], axis=1, inplace=True)
    else:
        new_df.drop(['text_cleaned','text','id'], axis=1, inplace=True)
        
    df.drop(['text_cleaned'],axis=1,inplace=True)
    return new_df.values
    
print('Processing train...')
train_hand_features = extract_features(train_df,train_flag=True)
print('Processing test...')
test_hand_features = extract_features(test_df)
print(train_hand_features.shape,test_hand_features.shape)
print(train_df.columns)

Processing train...


100%|██████████| 26/26 [08:55<00:00, 20.61s/it]


Processing test...


100%|██████████| 26/26 [04:02<00:00,  9.34s/it]


(19579, 26685) (8392, 26685)
Index(['id', 'text', 'author', 'n_.', 'n_...', 'n_,', 'n_:', 'n_;', 'n_-',
       'n_?', 'n_!', 'n_'', 'n_"', 'n_The ', 'n_I ', 'n_It ', 'n_He ', 'n_Me ',
       'n_She ', 'n_We ', 'n_They ', 'n_You ', 'n_the', 'n_ a ', 'n_appear',
       'n_little', 'n_was ', 'n_one ', 'n_two ', 'n_three ', 'n_ten ', 'n_is ',
       'n_are ', 'n_ed', 'n_however', 'n_ to ', 'n_into', 'n_about ', 'n_th',
       'n_er', 'n_ex', 'n_an ', 'n_ground', 'n_any', 'n_silence', 'n_wall'],
      dtype='object')


In [3]:
# https://spacy.io/usage/models#usage-import
# https://spacy.io/usage/models
import en_core_web_sm
spacy_nlp = en_core_web_sm.load()

# change ne to tag
def get_spacy_text(s):
    pos,tag,dep = '','',''
    for token in spacy_nlp(s):
        pos = pos + ' ' + token.pos_
        tag = tag + ' ' + token.tag_
        dep = dep + ' ' + token.dep_

    return pos,tag,dep

print(get_spacy_text('this is kaggle spooky games.'))

import time
start_t = time.time()
poss,tags,deps = [],[],[]
for s in train_df["text"].values:
    pos,tag,dep = get_spacy_text(s)
    poss.append(pos)
    tags.append(tag)
    deps.append(dep)
train_df['pos_txt'],train_df['tag_txt'],train_df['dep_txt'] = poss, tags, deps
print('train done',time.time() - start_t)


start_t = time.time()
poss,tags,deps = [],[],[]
for s in test_df["text"].values:
    pos,tag,dep = get_spacy_text(s)
    poss.append(pos)
    tags.append(tag)
    deps.append(dep)
test_df['pos_txt'],test_df['tag_txt'],test_df['dep_txt'] = poss, tags, deps
print('test done', time.time() - start_t)

(' DET VERB NOUN ADJ NOUN PUNCT', ' DT VBZ NN JJ NNS .', ' nsubj ROOT nmod amod attr punct')
train done 396.3616397380829
test done 188.14431023597717


In [4]:
# cnt on tag
c_vec3 = CountVectorizer(lowercase=False,ngram_range=(1,1))
c_vec3.fit(train_df['tag_txt'].values.tolist() + test_df['tag_txt'].values.tolist())
train_cvec3 = c_vec3.transform(train_df['tag_txt'].values.tolist()).toarray()
test_cvec3 = c_vec3.transform(test_df['tag_txt'].values.tolist()).toarray()
print(train_cvec3.shape,test_cvec3.shape)

# cnt on ne
c_vec4 = CountVectorizer(lowercase=False,ngram_range=(1,2))
c_vec4.fit(train_df['pos_txt'].values.tolist() + test_df['pos_txt'].values.tolist())
train_cvec4 = c_vec4.transform(train_df['pos_txt'].values.tolist()).toarray()
test_cvec4 = c_vec4.transform(test_df['pos_txt'].values.tolist()).toarray()
print(train_cvec4.shape,test_cvec4.shape)

# cnt on dep
c_vec7 = CountVectorizer(lowercase=False,ngram_range=(1,1))
c_vec7.fit(train_df['dep_txt'].values.tolist() + test_df['dep_txt'].values.tolist())
train_cvec7 = c_vec7.transform(train_df['dep_txt'].values.tolist()).toarray()
test_cvec7 = c_vec7.transform(test_df['dep_txt'].values.tolist()).toarray()
print(train_cvec7.shape,test_cvec7.shape)

# tfidf on tag
tf_vec5 = TfidfVectorizer(lowercase=False,ngram_range=(1,1))
tf_vec5.fit(train_df['tag_txt'].values.tolist() + test_df['tag_txt'].values.tolist())
train_tf5 = tf_vec5.transform(train_df['tag_txt'].values.tolist()).toarray()
test_tf5 = tf_vec5.transform(test_df['tag_txt'].values.tolist()).toarray()
print(train_tf5.shape,test_tf5.shape)

# tfidf on ne
tf_vec6 = TfidfVectorizer(lowercase=False,ngram_range=(1,2))
tf_vec6.fit(train_df['pos_txt'].values.tolist() + test_df['pos_txt'].values.tolist())
train_tf6 = tf_vec6.transform(train_df['pos_txt'].values.tolist()).toarray()
test_tf6 = tf_vec6.transform(test_df['pos_txt'].values.tolist()).toarray()
print(train_tf6.shape,test_tf6.shape)

# tfidf on dep
tf_vec8 = TfidfVectorizer(lowercase=False,ngram_range=(1,1))
tf_vec8.fit(train_df['dep_txt'].values.tolist() + test_df['dep_txt'].values.tolist())
train_tf8 = tf_vec8.transform(train_df['dep_txt'].values.tolist()).toarray()
test_tf8 = tf_vec8.transform(test_df['dep_txt'].values.tolist()).toarray()
print(train_tf8.shape,test_tf8.shape)

(19579, 38) (8392, 38)
(19579, 186) (8392, 186)
(19579, 45) (8392, 45)
(19579, 38) (8392, 38)
(19579, 186) (8392, 186)
(19579, 45) (8392, 45)


In [5]:
all_nlp_train = np.hstack([train_cvec3,train_cvec4,train_tf5,train_tf6,train_cvec7, train_tf8]) 
all_nlp_test = np.hstack([test_cvec3,test_cvec4,test_tf5,test_tf6, test_cvec7, test_tf8]) 
print('nlp feat done')

nlp feat done


In [6]:
eng_stopwords = [
    "a", "about", "above", "across", "after", "afterwards", "again", "against",
    "all", "almost", "alone", "along", "already", "also", "although", "always",
    "am", "among", "amongst", "amoungst", "amount", "an", "and", "another",
    "any", "anyhow", "anyone", "anything", "anyway", "anywhere", "are",
    "around", "as", "at", "back", "be", "became", "because", "become",
    "becomes", "becoming", "been", "before", "beforehand", "behind", "being",
    "below", "beside", "besides", "between", "beyond", "bill", "both",
    "bottom", "but", "by", "call", "can", "cannot", "cant", "co", "con",
    "could", "couldnt", "cry", "de", "describe", "detail", "do", "done",
    "down", "due", "during", "each", "eg", "eight", "either", "eleven", "else",
    "elsewhere", "empty", "enough", "etc", "even", "ever", "every", "everyone",
    "everything", "everywhere", "except", "few", "fifteen", "fifty", "fill",
    "find", "fire", "first", "five", "for", "former", "formerly", "forty",
    "found", "four", "from", "front", "full", "further", "get", "give", "go",
    "had", "has", "hasnt", "have", "he", "hence", "her", "here", "hereafter",
    "hereby", "herein", "hereupon", "hers", "herself", "him", "himself", "his",
    "how", "however", "hundred", "i", "ie", "if", "in", "inc", "indeed",
    "interest", "into", "is", "it", "its", "itself", "keep", "last", "latter",
    "latterly", "least", "less", "ltd", "made", "many", "may", "me",
    "meanwhile", "might", "mill", "mine", "more", "moreover", "most", "mostly",
    "move", "much", "must", "my", "myself", "name", "namely", "neither",
    "never", "nevertheless", "next", "nine", "no", "nobody", "none", "noone",
    "nor", "not", "nothing", "now", "nowhere", "of", "off", "often", "on",
    "once", "one", "only", "onto", "or", "other", "others", "otherwise", "our",
    "ours", "ourselves", "out", "over", "own", "part", "per", "perhaps",
    "please", "put", "rather", "re", "same", "see", "seem", "seemed",
    "seeming", "seems", "serious", "several", "she", "should", "show", "side",
    "since", "sincere", "six", "sixty", "so", "some", "somehow", "someone",
    "something", "sometime", "sometimes", "somewhere", "still", "such",
    "system", "take", "ten", "than", "that", "the", "their", "them",
    "themselves", "then", "thence", "there", "thereafter", "thereby",
    "therefore", "therein", "thereupon", "these", "they", "thick", "thin",
    "third", "this", "those", "though", "three", "through", "throughout",
    "thru", "thus", "to", "together", "too", "top", "toward", "towards",
    "twelve", "twenty", "two", "un", "under", "until", "up", "upon", "us",
    "very", "via", "was", "we", "well", "were", "what", "whatever", "when",
    "whence", "whenever", "where", "whereafter", "whereas", "whereby",
    "wherein", "whereupon", "wherever", "whether", "which", "while", "whither",
    "who", "whoever", "whole", "whom", "whose", "why", "will", "with",
    "within", "without", "would", "yet", "you", "your", "yours", "yourself",
    "yourselves"]

In [7]:
# replace
# train_df['text'] = train_df['text'].str.replace('[^a-zA-Z0-9]', ' ')
# test_df['text'] =test_df['text'].str.replace('[^a-zA-Z0-9]', ' ')

## Number of words in the text ##
train_df["num_words"] = train_df["text"].apply(lambda x: len(str(x).split()))
test_df["num_words"] = test_df["text"].apply(lambda x: len(str(x).split()))

## Number of unique words in the text ##
train_df["num_unique_words"] = train_df["text"].apply(lambda x: len(set(str(x).split())))
test_df["num_unique_words"] = test_df["text"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the text ##
train_df["num_chars"] = train_df["text"].apply(lambda x: len(str(x)))
test_df["num_chars"] = test_df["text"].apply(lambda x: len(str(x)))

## Number of stopwords in the text ##
train_df["num_stopwords"] = train_df["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))
test_df["num_stopwords"] = test_df["text"].apply(lambda x: len([w for w in str(x).lower().split() if w in eng_stopwords]))

## Number of punctuations in the text ##
import string
train_df["num_punctuations"] =train_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
test_df["num_punctuations"] =test_df['text'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the text ##
train_df["num_words_upper"] = train_df["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
test_df["num_words_upper"] = test_df["text"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the text ##
train_df["num_words_title"] = train_df["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
test_df["num_words_title"] = test_df["text"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the text ##
train_df["mean_word_len"] = train_df["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
test_df["mean_word_len"] = test_df["text"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

# add features
def add_feat(df):
    df['unique_r'] = df['num_unique_words'] / df['num_words']
    df['w_p'] = df['num_words'] - df['num_punctuations']
    df['w_p_r'] = df['w_p'] / df['num_words']
    df['stop_r'] = df['num_stopwords'] / df['num_words']
    df['w_p_stop'] = df['w_p'] - df['num_stopwords']
    df['w_p_stop_r'] = df['w_p_stop'] / df['num_words']
    df['num_words_upper_r'] = df['num_words_upper'] / df['num_words']
    df['num_words_title_r'] = df['num_words_title'] / df['num_words']

add_feat(train_df)
add_feat(test_df)
print(train_df.columns)

Index(['id', 'text', 'author', 'n_.', 'n_...', 'n_,', 'n_:', 'n_;', 'n_-',
       'n_?', 'n_!', 'n_'', 'n_"', 'n_The ', 'n_I ', 'n_It ', 'n_He ', 'n_Me ',
       'n_She ', 'n_We ', 'n_They ', 'n_You ', 'n_the', 'n_ a ', 'n_appear',
       'n_little', 'n_was ', 'n_one ', 'n_two ', 'n_three ', 'n_ten ', 'n_is ',
       'n_are ', 'n_ed', 'n_however', 'n_ to ', 'n_into', 'n_about ', 'n_th',
       'n_er', 'n_ex', 'n_an ', 'n_ground', 'n_any', 'n_silence', 'n_wall',
       'pos_txt', 'tag_txt', 'dep_txt', 'num_words', 'num_unique_words',
       'num_chars', 'num_stopwords', 'num_punctuations', 'num_words_upper',
       'num_words_title', 'mean_word_len', 'unique_r', 'w_p', 'w_p_r',
       'stop_r', 'w_p_stop', 'w_p_stop_r', 'num_words_upper_r',
       'num_words_title_r'],
      dtype='object')


In [8]:
## Prepare the data for modeling ###
author_mapping_dict = {'EAP':0, 'HPL':1, 'MWS':2}
train_y = train_df['author'].map(author_mapping_dict)
train_id = train_df['id'].values
test_id = test_df['id'].values

## add tfidf and svd
tfidf_vec = TfidfVectorizer(ngram_range=(1,3), max_df=0.8,lowercase=False, sublinear_tf=True)
full_tfidf = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf = tfidf_vec.transform(test_df['text'].values.tolist())

print(train_tfidf.shape,test_tfidf.shape)

# svd1
n_comp = 30
svd_obj = TruncatedSVD(n_components=n_comp)
svd_obj.fit(full_tfidf)
train_svd = pd.DataFrame(svd_obj.transform(train_tfidf))
test_svd = pd.DataFrame(svd_obj.transform(test_tfidf))
print(train_svd.shape,test_svd.shape)

## add tfidf char
tfidf_vec = TfidfVectorizer(ngram_range=(3,7), analyzer='char',max_df=0.8, sublinear_tf=True)
full_tfidf2 = tfidf_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_tfidf2 = tfidf_vec.transform(train_df['text'].values.tolist())
test_tfidf2 = tfidf_vec.transform(test_df['text'].values.tolist())
print(train_tfidf2.shape,test_tfidf2.shape)

## add svd2
svd_obj = TruncatedSVD(n_components=n_comp)
svd_obj.fit(full_tfidf2)
train_svd2 = pd.DataFrame(svd_obj.transform(train_tfidf2))
test_svd2 = pd.DataFrame(svd_obj.transform(test_tfidf2))
print(train_svd2.shape,test_svd2.shape)


## add cnt vec
c_vec = CountVectorizer(ngram_range=(1,3),max_df=0.8, lowercase=False)
full_cvec1 = c_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_cvec = c_vec.transform(train_df['text'].values.tolist())
test_cvec = c_vec.transform(test_df['text'].values.tolist())
print(train_cvec.shape,test_cvec.shape)

## add svd3
svd_obj = TruncatedSVD(n_components=n_comp)
svd_obj.fit(full_cvec1)
train_svd3 = pd.DataFrame(svd_obj.transform(train_cvec))
test_svd3 = pd.DataFrame(svd_obj.transform(test_cvec))

# add cnt char
c_vec = CountVectorizer(ngram_range=(3,7), analyzer='char',max_df=0.8)
full_cvec2 = c_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_cvec2 = c_vec.transform(train_df['text'].values.tolist())
test_cvec2 = c_vec.transform(test_df['text'].values.tolist())
print(train_cvec2.shape,test_cvec2.shape)

## add svd4
svd_obj = TruncatedSVD(n_components=n_comp)
svd_obj.fit(full_cvec2)
train_svd4 = pd.DataFrame(svd_obj.transform(train_cvec2))
test_svd4 = pd.DataFrame(svd_obj.transform(test_cvec2))

(19579, 885129) (8392, 885129)
(19579, 30) (8392, 30)
(19579, 1354551) (8392, 1354551)
(19579, 30) (8392, 30)
(19579, 885129) (8392, 885129)
(19579, 1354551) (8392, 1354551)


In [9]:
# add cnt char
c_vec = CountVectorizer(ngram_range=(1,2), analyzer='char',max_df=0.8)
full_cvec3 = c_vec.fit_transform(train_df['text'].values.tolist() + test_df['text'].values.tolist())
train_cvec3 = c_vec.transform(train_df['text'].values.tolist())
test_cvec3 = c_vec.transform(test_df['text'].values.tolist())
print(train_cvec3.shape,test_cvec3.shape)

(19579, 1068) (8392, 1068)


In [10]:
all_svd_train = np.hstack([train_svd,train_svd2,train_svd3,train_svd4,train_cvec3.toarray()])
all_svd_test = np.hstack([test_svd,test_svd2,test_svd3,test_svd4,test_cvec3.toarray()])

In [13]:
# add naive feature
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import KFold
from sklearn.metrics import log_loss

feat_cnt = 5
train_Y = train_y

def gen_nb_feats(rnd=1):
    help_tfidf_train,help_tfidf_test = np.zeros((19579,3)),np.zeros((8392,3))
    help_tfidf_train2,help_tfidf_test2 = np.zeros((19579,3)),np.zeros((8392,3))
    help_cnt1_train,help_cnt1_test = np.zeros((19579,3)),np.zeros((8392,3))
    help_cnt2_train,help_cnt2_test = np.zeros((19579,3)),np.zeros((8392,3))
    hand_train, hand_test = np.zeros((19579,3)),np.zeros((8392,3))

    kf = KFold(n_splits=feat_cnt, shuffle=True, random_state=23*rnd)
    for train_index, test_index in kf.split(train_tfidf):
        # tfidf to nb
        X_train, X_test = train_tfidf[train_index], train_tfidf[test_index]
        y_train, y_test = train_Y[train_index], train_Y[test_index]
        tmp_model = MultinomialNB(alpha=0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_tfidf)
        help_tfidf_train[test_index] = tmp_train_feat
        help_tfidf_test += tmp_test_feat/feat_cnt

        # tfidf to nb
        X_train, X_test = train_tfidf2[train_index], train_tfidf2[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_tfidf2)
        help_tfidf_train2[test_index] = tmp_train_feat
        help_tfidf_test2 += tmp_test_feat/feat_cnt

        # count vec to nb
        X_train, X_test = train_cvec[train_index], train_cvec[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_cvec)
        help_cnt1_train[test_index] = tmp_train_feat
        help_cnt1_test += tmp_test_feat/feat_cnt

        # count vec2 to nb 
        X_train, X_test = train_cvec2[train_index], train_cvec2[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_cvec2)
        help_cnt2_train[test_index] = tmp_train_feat
        help_cnt2_test += tmp_test_feat/feat_cnt
        
        # hand feature to nb
        X_train, X_test = train_hand_features[train_index], train_hand_features[test_index]
        tmp_model = MultinomialNB(0.025,fit_prior=False)
        tmp_model.fit(X_train,y_train)
        tmp_train_feat = tmp_model.predict_proba(X_test)
        tmp_test_feat = tmp_model.predict_proba(test_hand_features)
        hand_train[test_index] = tmp_train_feat
        hand_test += tmp_test_feat/feat_cnt
    
    help_train_feat = np.hstack([help_tfidf_train,help_tfidf_train2,help_cnt1_train,help_cnt2_train,hand_train])
    help_test_feat = np.hstack([help_tfidf_test,help_tfidf_test2,help_cnt1_test,help_cnt2_test,hand_test])

    return help_train_feat,help_test_feat

help_train_feat,help_test_feat = gen_nb_feats(1)
print(help_train_feat.shape,help_test_feat.shape)
help_train_feat2,help_test_feat2 = gen_nb_feats(2)
help_train_feat3,help_test_feat3 = gen_nb_feats(3)

(19579, 15) (8392, 15)


In [14]:
import pickle
with open('nn_feat.pkl','rb') as fin:
    nn_train1,nn_test1 = pickle.load(fin)
with open('nn_2gram_feat.pkl','rb') as fin:
    nn_train2,nn_test2 = pickle.load(fin)
print('done')

done


In [15]:
# combine feats
cols_to_drop = ['id','text','tag_txt','pos_txt','dep_txt']
train_X = train_df.drop(cols_to_drop+['author'], axis=1).values
test_X = test_df.drop(cols_to_drop, axis=1).values
train_X = np.hstack([train_X, all_svd_train, all_nlp_train])
test_X = np.hstack([test_X, all_svd_test, all_nlp_test])

f_train_X = np.hstack([train_X, help_train_feat,help_train_feat2,help_train_feat3,nn_train1,nn_train2])
#f_train_X = np.round(f_train_X,4)
f_test_X = np.hstack([test_X, help_test_feat,help_test_feat2,help_test_feat3,nn_test1,nn_test2])
#f_test_X = np.round(f_test_X,4)
print(f_train_X.shape, f_test_X.shape)

(19579, 1866) (8392, 1866)


In [16]:
with open('feat.pkl','wb') as fout:
    pickle.dump([f_train_X,f_test_X],fout)
print('dump for xgb')

dump for xgb


In [17]:
from sklearn.model_selection import StratifiedKFold
def cv_test(k_cnt=3, s_flag = False):
    rnd = 420
    if s_flag:
        kf = StratifiedKFold(n_splits=k_cnt, shuffle=True, random_state=rnd)
    else:
        kf = KFold(n_splits=k_cnt, shuffle=True, random_state=rnd)
    test_pred = None
    weighted_test_pred = None
    org_train_pred = None
    avg_k_score = 0
    reverse_score = 0
    best_loss = 100
    best_single_pred = None
    for train_index, test_index in kf.split(f_train_X,train_Y):
        X_train, X_test = f_train_X[train_index], f_train_X[test_index]
        y_train, y_test = train_Y[train_index], train_Y[test_index]
        params = {
                'colsample_bytree': 0.7,
                'subsample': 0.8,
                'eta': 0.04,
                'max_depth': 3,
                'eval_metric':'mlogloss',
                'objective':'multi:softprob',
                'num_class':3,
                }
        
        # def mat
        d_train = xgb.DMatrix(X_train, y_train)
        d_valid = xgb.DMatrix(X_test, y_test)
        d_test = xgb.DMatrix(f_test_X)
        
        watchlist = [(d_train, 'train'), (d_valid, 'valid')]
        # train model
        m = xgb.train(params, d_train, 2000, watchlist, 
                        early_stopping_rounds=200,
                        verbose_eval=200)
        
        # get res
        train_pred = m.predict(d_train)
        valid_pred = m.predict(d_valid)
        tmp_train_pred = m.predict(xgb.DMatrix(f_train_X))
        
        # cal score
        train_score = log_loss(y_train,train_pred)
        valid_score = log_loss(y_test,valid_pred)
        print('train log loss',train_score,'valid log loss',valid_score)
        avg_k_score += valid_score
        rev_valid_score = 1.0/valid_score # use for weighted
        reverse_score += rev_valid_score # sum
        print('rev',rev_valid_score)
        
        if test_pred is None:
            test_pred = m.predict(d_test)
            weighted_test_pred = test_pred*rev_valid_score
            org_train_pred = tmp_train_pred
            best_loss = valid_score
            best_single_pred = test_pred
        else:
            curr_pred = m.predict(d_test)
            test_pred += curr_pred
            weighted_test_pred += curr_pred*rev_valid_score # fix bug here
            org_train_pred += tmp_train_pred
            # find better single model
            if valid_score < best_loss:
                print('BETTER')
                best_loss = valid_score
                best_single_pred = curr_pred

    # avg
    test_pred = test_pred / k_cnt
    #test_pred = np.round(test_pred,4)
    org_train_pred = org_train_pred / k_cnt
    avg_k_score = avg_k_score/k_cnt

    submiss=pd.read_csv("./input/sample_submission.csv")
    submiss['EAP']=test_pred[:,0]
    submiss['HPL']=test_pred[:,1]
    submiss['MWS']=test_pred[:,2]
    submiss.to_csv("results/xgb_res_{}_{}.csv".format(k_cnt, s_flag),index=False)
    print(submiss.head(5))
    print('--------------')
    print(reverse_score)
    # weigthed
    submiss=pd.read_csv("./input/sample_submission.csv")
    weighted_test_pred = weighted_test_pred / reverse_score
    #weighted_test_pred = np.round(weighted_test_pred,4)
    submiss['EAP']=weighted_test_pred[:,0]
    submiss['HPL']=weighted_test_pred[:,1]
    submiss['MWS']=weighted_test_pred[:,2]
    submiss.to_csv("results/weighted_xgb_res_{}_{}.csv".format(k_cnt, s_flag),index=False)
    print(submiss.head(5))
    print('---------------')
    # best single
    submiss=pd.read_csv("./input/sample_submission.csv")
    submiss['EAP']=best_single_pred[:,0]
    submiss['HPL']=best_single_pred[:,1]
    submiss['MWS']=best_single_pred[:,2]
    submiss.to_csv("results/single_xgb_res_{}_{}.csv".format(k_cnt, s_flag),index=False)
    print(submiss.head(5))
    print('---------------')
    
    # train log loss
    print('local average valid loss',avg_k_score)
    print('train log loss', log_loss(train_Y,org_train_pred))
    
print('def done')

def done


In [18]:
cv_test(5, True)

[0]	train-mlogloss:1.05529	valid-mlogloss:1.05497
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 200 rounds.
[200]	train-mlogloss:0.225782	valid-mlogloss:0.263446
[400]	train-mlogloss:0.180428	valid-mlogloss:0.256579
[600]	train-mlogloss:0.149666	valid-mlogloss:0.256031
[800]	train-mlogloss:0.125709	valid-mlogloss:0.256907
Stopping. Best iteration:
[693]	train-mlogloss:0.13776	valid-mlogloss:0.255587

train log loss 0.116297053862 valid log loss 0.257254700976
rev 3.8871981589
[0]	train-mlogloss:1.05496	valid-mlogloss:1.05548
Multiple eval metrics have been passed: 'valid-mlogloss' will be used for early stopping.

Will train until valid-mlogloss hasn't improved in 200 rounds.
[200]	train-mlogloss:0.223538	valid-mlogloss:0.271343
[400]	train-mlogloss:0.179055	valid-mlogloss:0.262164
[600]	train-mlogloss:0.148997	valid-mlogloss:0.260695
[800]	train-mlogloss:0.12562	valid-mlogloss:0.260974
Stop

In [19]:
# cv_test(10, True)
# 276xx not good